<img src="images/ublogo.png"/>

### CSE610 - Bayesian Non-parametric Machine Learning

  - Lecture Notes
  - Instructor - Varun Chandola
  - Term - Fall 2020

In [1]:
import numpy as np
from scipy.stats import multivariate_normal as mvn
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
sns.set_style('whitegrid')
sns.set_context('paper',font_scale=2)
from mpl_toolkits import mplot3d
%config InlineBackend.print_figure_kwargs = {'bbox_inches':None}
%matplotlib inline

In [2]:
def bayesPosterior(X,y,sigmansq,Sigmap):
    Ainv = np.linalg.inv((1/sigmansq)*np.dot(X,X.T) + np.linalg.inv(Sigmap))
    wpost = (1/sigmansq)*np.dot(Ainv,np.dot(X,y))
    return wpost,Ainv
    

In [ ]:
def plot_contours(ax,mean,cov,limits=(-10,10),cm='Reds',levels=6):
    w1s = w2s = np.linspace(limits[0],limits[1],100)
    W1,W2 = np.meshgrid(w1s,w2s)

    pdfs = []
    for w1,w2 in zip(W1.flatten(),W2.flatten()):
        w = np.array([w1,w2])
        pdfs.append(mvn.pdf(w,mean,cov))
    pdfs = np.array(pdfs)
    pdfs = np.reshape(pdfs,W1.shape)

    cfset = ax.contourf(W1, W2, pdfs, levels=levels,cmap=cm)
    cset = ax.contour(W1, W2, pdfs, levels=levels, colors='k',alpha=0.6)
    ax.clabel(cset, inline=1, fontsize=10)